<a href="https://colab.research.google.com/github/gkianfar/TIHM-Dataset-Visualization/blob/main/tihm_featureExtraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Clone Github repo and set working directory path**

In [94]:
!git clone https://github.com/gkianfar/TIHM-Dataset-Visualization

Cloning into 'TIHM-Dataset-Visualization'...
remote: Enumerating objects: 69, done.
remote: Counting objects: 100% (69/69), done.
remote: Compressing objects: 100% (67/67), done.
remote: Total 69 (delta 32), reused 13 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (69/69), 14.10 MiB | 7.28 MiB/s, done.
Resolving deltas: 100% (32/32), done.


**Load packages**

In [95]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from matplotlib import cm
import matplotlib.patches as mpatches
from collections.abc import Iterable
from sklearn.preprocessing import StandardScaler
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import os
import warnings
warnings.filterwarnings('ignore')
from matplotlib import cm
#from utils import *

%matplotlib inline
## Set colour palette
ibm_colorblind = ['#648FFF', '#FE6100', '#DC267F', '#785EF0', '#FFB000','#48A9A6']
sns.set_palette(ibm_colorblind)

**Functions**

In [96]:


def correct_col_type(df,col):
    raw_type = str(type(df[col].dtype)).split('.')[-1].split('\'')[0].lower()
    #print(col,raw_type)
    if 'object' in raw_type:
        if 'date' in col or 'timestamp' in col or 'datetime' in col:
            return pd.to_datetime(df[col])
        else:
            return df[col].astype('category')
    else:
        return df[col]


def gen_date_col(df, tcol):
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['date'] = df[tcol].dt.date
    return df





**Set Data and Fig folders paths**

In [112]:
## Please change the path with the path of your dataset
%cd /content/TIHM-Dataset-Visualization
DPATH = './Data'
SAVE_PATH = './Figs/'
if not os.path.exists(SAVE_PATH):
    os.makedirs(SAVE_PATH)



/content/TIHM-Dataset-Visualization


In [113]:
delta = 'sec' # minute
interval = 6 #hours
if delta == 'sec':
  interval = interval*3600
elif delta == 'min':
  interval = interval*60





**Import Files**

In [114]:
### Labels
f = 'Labels.csv'
fpth = os.path.join(DPATH,f)
labels_df = pd.read_csv(fpth)
for col in labels_df.columns:
    labels_df[col] = correct_col_type(labels_df,col)
if 'date' in labels_df.columns:
    labels_df = labels_df.rename(columns={'date':'timestamp'})
    labels_df['timestamp'] = pd.to_datetime(labels_df['timestamp'])
    labels_df['date'] = pd.to_datetime(labels_df['timestamp'].dt.date)
if 'type' in labels_df.columns:
    labels_df = labels_df.rename(columns={'type':'label'})



labels_patients = np.unique(labels_df['patient_id'])

Agitation_labels = labels_df.loc[labels_df['label'] == 'Agitation']
Agitation_labels

,patient_id,timestamp,label,date
2,16f4b,2019-04-11 12:00:22,Agitation,2019-04-11
4,16f4b,2019-04-14 12:00:07,Agitation,2019-04-14
5,16f4b,2019-04-15 18:00:24,Agitation,2019-04-15
6,16f4b,2019-04-16 18:00:38,Agitation,2019-04-16
11,16f4b,2019-04-21 12:00:55,Agitation,2019-04-21
...,...,...,...,...
587,0d5ef,2019-06-28 18:02:11,Agitation,2019-06-28
588,d7a46,2019-06-28 18:02:21,Agitation,2019-06-28
598,6b29b,2019-06-29 12:00:27,Agitation,2019-06-29
599,95899,2019-06-29 12:01:20,Agitation,2019-06-29


In [119]:
### Activity

f = 'Activity.csv'
fpth = os.path.join(DPATH,f)
df = pd.read_csv(fpth)
for col in df.columns:
    df[col] = correct_col_type(df,col)
if 'date' in df.columns:
    activity_df = df.rename(columns={'date':'timestamp'})
    activity_df['timestamp'] = pd.to_datetime(activity_df['timestamp'])
    activity_df['date'] = pd.to_datetime(activity_df['timestamp'].dt.date)
    activity_df['time'] = activity_df['timestamp'].dt.time


activity_patients = np.unique(activity_df['patient_id'])

In [116]:
### Physiology
f = 'Physiology.csv'
fpth = os.path.join(DPATH,f)
df = pd.read_csv(fpth)
for col in df.columns:
    df[col] = correct_col_type(df,col)
if 'date' in df.columns:
    physiology_df = df.rename(columns={'date':'timestamp'})
    physiology_df['timestamp'] = pd.to_datetime(physiology_df['timestamp'])
    physiology_df['date'] = pd.to_datetime(physiology_df['timestamp'].dt.date)
#physiology_df = gen_date_col(df,tcol='timestamp')

physiology_patients = np.unique(physiology_df['patient_id'])

In [117]:
### Sleep
f = 'Sleep.csv'
fpth = os.path.join(DPATH,f)
df = pd.read_csv(fpth)
for col in df.columns:
    df[col] = correct_col_type(df,col)
if 'date' in df.columns:
    sleep_df = df.rename(columns={'date':'timestamp'})
    sleep_df['timestamp'] = pd.to_datetime(sleep_df['timestamp'])
    sleep_df['date'] = pd.to_datetime(sleep_df['timestamp'].dt.date)
#physiology_df = gen_date_col(df,tcol='timestamp')

sleep_patients = np.unique(sleep_df['patient_id'])

In [142]:
interval = 6 # hours to filter activities
agitation_df = labels_df[labels_df['label']=='Agitation'].copy()


agitation_location = agitation_df[['patient_id', 'timestamp']]
agitation_patientID = agitation_df['patient_id'].tolist()
agitation_endTime = agitation_df['timestamp'].tolist()
### 6 hours earlier
agitation_startTime = pd.to_datetime(agitation_endTime) - pd.Timedelta(hours=interval)
agitation_startTime = agitation_startTime.tolist()

### patients that have agitation and the number of samples that each has
agitation_patients = np.unique(agitation_location['patient_id'])
each_patient_agitation_sample = agitation_df['patient_id'].value_counts()

# store the agitation days for each paient
agitation_days = {}
for id in agitation_patients:
   agitation_days[id] = agitation_df['date'][agitation_df['patient_id'] == id].unique()

activity_days = {}
for id in agitation_patients:
   activity_days[id] = activity_df['date'][activity_df['patient_id'] == id].unique()


agitation_dataset = []
# Iterate through each patient_id, start_time, and end_time
for index, (patient_id, start_time, end_time) in enumerate(zip(agitation_patientID, agitation_startTime, agitation_endTime)):
    #print(f'{index}-user: {patient_id} - from {start_time} to {end_time}')
    # Filter the DataFrame for the current patient and time range
    filtered_df = activity_df[(activity_df['patient_id'] == patient_id) &
                            (activity_df['timestamp'] >= start_time) &
                            (activity_df['timestamp'] <= end_time)]

    # Store the results in the list
    agit_activity = filtered_df['location_name'].tolist()

    patient_agit_days = agitation_days[patient_id]
    patient_activity_days = activity_days[patient_id]
    pateint_non_agit_days = list(set(patient_activity_days) - set(patient_agit_days))

    #print(f'nin agit len: {len(pateint_non_agit_days)}')

    # separate the corresponding non agitation intervals from normal days of each patient
    non_agit_activity = []
    for day in pateint_non_agit_days:
      if start_time.time() < end_time.time():
        filtered_df = activity_df[(activity_df['patient_id'] == patient_id) &
                                  (activity_df['date'] == day) &
                                  (activity_df['time'] >= start_time.time()) &
                                  (activity_df['time'] <= end_time.time())]
      else:
        raise Exception("Start time is after end time")


      filtered_activity = filtered_df['location_name'].tolist()
      non_agit_activity.append(filtered_activity)
      #print(f'day {day} - {filtered_df}')
    agitation_dataset.append([patient_id, start_time, end_time, agit_activity, non_agit_activity])
## save date in a class object
## the data from the same patient, the same period in other days



In [146]:
class AGITATION_DATASET():
  def __init__(self,agitation_dataset):
    self.patient_id = agitation_dataset[:][0]
    self.start_time = agitation_dataset[:][1]
    self.end_time = agitation_dataset[:][2]
    self.agitation_activity = agitation_dataset[:][3]
    self.non_agitation = agitation_dataset[:][4]
agitation_dataset_obj = AGITATION_DATASET(agitation_dataset)



In [92]:
# Mapping function for all categories in Location_name column

activity_locations_unique = activity_df['location_name'].unique()
activity_locations_map = {location: index for index, location in enumerate(activity_locations_unique)}

activity_change_count = []
activity_occurance_count = []
for filtered_activity in agitation_activity:

  dummy_filtered_activity = pd.Series(filtered_activity)
  activity_occurance_count.append(dummy_filtered_activity.value_counts())

  dummy_filtered_activity = dummy_filtered_activity.map(activity_locations_map)
  change_count = dummy_filtered_activity.diff().value_counts()

  if any(change_count.index == 0):
    unchanged_count = change_count[change_count.index == 0].values[0]
  else :
    unchanged_count = 0

  # Counting number of changes between locations by substracting the total number of unchanged locations from the max possible changes
  activity_change_count.append((dummy_filtered_activity.shape[0]-1)-unchanged_count)



In [71]:
def filter_similar_intervals(x, end_time, interval=6):

    start_time = end_time - pd.Timedelta(hours=interval)
    if start_time <= end_time:
        # Interval does not span midnight
        return start_time <= x <= end_time
    else:
        # Interval spans midnight
        return x >= start_time or x <= end_time

# Given a time check if withing the future interval any agitation label exists (True: no agitation, False: existing agitation)
def filter_no_agitation_label(x, agitation_df, interval):
    end_time = x + pd.Timedelta(hours=interval)
    if agitation_df['timestamp'].between(x, end_time).any():
        return False
    else:
        return True

for patient_id, start_time, end_time in zip(agitation_patientID, agitation_startTime, agitation_endTime):
    # Filter the DataFrame for the current patient and time range
    filtered_df = activity_df[(activity_df['patient_id'] == patient_id)]
    filtered_df['similar_interval'] = filtered_df[filtered_df['timestamp'].apply(lambda x: filter_similar_intervals(x, end_time, interval))]


    # Store the results in the list
    filtered_activity = filtered_df['location_name'].tolist()

135

In [93]:
labels_df['timestamp'].dt.time()

TypeError: 'Series' object is not callable